In [36]:
import flyte
import flyte.remote as remote

import asyncio
from typing import List

In [37]:
flyte.init_from_config("../../config.yaml")

In [38]:
env = flyte.TaskEnvironment(
    name="env",
    image=flyte.Image
        .from_debian_base()
        .with_pip_packages("pandas", "requests"),
)

In [39]:
@env.task()
async def hello_worker(id: int) -> str:
    print(f"hello, my id is: {id} and I am being run by Action: {flyte.ctx().action}")
    return str(id)


In [40]:
@env.task()
async def hello_driver(ids: List[int] = [1, 2, 3]) -> List[str]:
    coros = []
    with flyte.group("fanout-group"):
        for id in ids:
            coros.append(hello_worker(id))
            print(f"working with id: {id}")

        vals = await asyncio.gather(*coros)

    return vals

In [41]:
run = await flyte.run.aio(hello_driver, ids=[1, 2, 3, 4, 5])

12:31:39.464309 WARNING  _deploy.py:159 - Built Image for environment env, image:                                                                               
                         356633062068.dkr.ecr.us-east-2.amazonaws.com/union/demo:flyte-d57df1157dcd010a7c4cd3e839a347ee

In [42]:
run.url

'https://demo.hosted.unionai.cloud/v2/runs/project/flytesnacks/domain/development/r9wvncwtz2v42qs6sgv6'

In [43]:
run.name

'r9wvncwtz2v42qs6sgv6'

In [44]:
await run.show_logs()

 working with id: 1
 working with id: 2
 working with id: 3
 working with id: 4
 working with id: 5

Scrolled 5 lines of logs.

In [45]:
run.details()

RunDetails(
    labels='',
    annotations='',
    env-vars='',
    is-interruptible='',
    cache-overwrite=False,
    name='r9wvncwtz2v42qs6sgv6',
    start_time='2025-08-12T19:31:41.367878+00:00',
    end_time='2025-08-12T19:32:01+00:00',
    run_time='19 secs',
    phase='PHASE_SUCCEEDED',
    error='NA',
    task='env.hello_driver',
    task_type='python',
    task_version='f22410696164f8d18207430cb695804d',
    attempts=[],
    error='NA',
    phase='PHASE_SUCCEEDED',
    group='',
    parent=''
)

In [46]:
actions = [a.name for a in remote.Action.listall(for_run_name=run.name)]

In [47]:
actions

['a0',
 'bbnt6kwprld4km8wi753mtvnu',
 '1u021mxgkx0tsdwtrljvg9no4',
 '6kklgb0dpm57n65d77cvejyaw',
 'l7arn24yjcsfcx3ozqgckfq7',
 '9sphagvd8vz9lodhviwa4x0a2']

In [54]:
await remote.Action.get(run_name="r9wvncwtz2v42qs6sgv6", name="bbnt6kwprld4km8wi753mtvnu").show_logs(raw=True)

 hello, my id is: 1 and I am being run by Action: ActionID('bbnt6kwprld4km8wi753mtvnu', 
run_name='r9wvncwtz2v42qs6sgv6', project='flytesnacks', domain='development', org='demo')

In [55]:
for action in remote.Action.listall(for_run_name=run.name):
    await remote.Action.get(run_name="r9wvncwtz2v42qs6sgv6", name=action.name).show_logs(raw=True)

 working with id: 1

 working with id: 2

 working with id: 3

 working with id: 4

 working with id: 5

 hello, my id is: 1 and I am being run by Action: ActionID('bbnt6kwprld4km8wi753mtvnu', 
run_name='r9wvncwtz2v42qs6sgv6', project='flytesnacks', domain='development', org='demo')

 hello, my id is: 3 and I am being run by Action: ActionID('1u021mxgkx0tsdwtrljvg9no4', 
run_name='r9wvncwtz2v42qs6sgv6', project='flytesnacks', domain='development', org='demo')

 hello, my id is: 2 and I am being run by Action: ActionID('6kklgb0dpm57n65d77cvejyaw', 
run_name='r9wvncwtz2v42qs6sgv6', project='flytesnacks', domain='development', org='demo')

 hello, my id is: 4 and I am being run by Action: ActionID('l7arn24yjcsfcx3ozqgckfq7', 
run_name='r9wvncwtz2v42qs6sgv6', project='flytesnacks', domain='development', org='demo')

 hello, my id is: 5 and I am being run by Action: ActionID('9sphagvd8vz9lodhviwa4x0a2', 
run_name='r9wvncwtz2v42qs6sgv6', project='flytesnacks', domain='development', org='demo')